## AES-GCM Encryption Example in Python

In this example, we use **AES in Galois/Counter Mode (GCM)** for authenticated encryption. 
GCM provides both confidentiality (encryption) and integrity/authentication (via a tag).

We will:
1. Generate an AES-128 key.
2. Generate a random nonce (IV) for encryption.
3. Define a plaintext message and optional additional authenticated data (AAD).
4. Encrypt the message using AES-GCM.


In [13]:
# Import required libraries
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import numpy as np
from os import urandom
#import sympy as sp
from IPython.display import display
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

### 1. Generate AES-128 key

AES-128 requires a 128-bit key (16 bytes).  
We use the `AESGCM.generate_key` function to generate a secure random key.


In [14]:
key = AESGCM.generate_key(bit_length=128)  # 128-bit key = 16 bytes
aesgcm = AESGCM(key)  # Initialize AES-GCM object with the key
print("Key:", key.hex())

Key: 53ecdbe519bb4f77255acfb496f9cc13


### 2. Generate a random nonce (IV)

GCM mode requires a unique nonce for each encryption with the same key.  
The recommended nonce size for GCM is 96 bits (12 bytes). We generate it using `urandom`.


In [15]:
nonce = urandom(12)  # 12-byte random nonce

### 3. Define plaintext and optional Associated Authenticated Data (AAD)

- `plaintext` is the message we want to encrypt.
- `aad` is optional data that is authenticated but **not encrypted**.  
  This ensures that any tampering with AAD will be detected during decryption.


In [16]:
plaintext = b"A un amigo perdido"
aad = b"authenticated-but-not-encrypted"

### 4. Encrypt the plaintext

We use the `encrypt` method of `AESGCM`, passing:
- `nonce`: the random IV,
- `plaintext`: the message to encrypt,
- `aad`: optional associated data.

The result is a ciphertext that **includes the authentication tag** at the end.


In [17]:
ciphertext = aesgcm.encrypt(nonce, plaintext, aad)
print("Ciphertext (hex):", ciphertext.hex())

Ciphertext (hex): 4f43cb6dd1be57a68ba8eb01801946169f0611999629433f801d209cfc1343e3ba55


* Key size: AES-128 is secure for most applications, but AES-256 can be used for higher post-quantum security.

* Nonce uniqueness: Never reuse the same nonce with the same key. Reusing nonces in GCM compromises security.

* AAD: Useful for including headers or metadata in the authentication check without encrypting them.

* Authentication tag: Automatically appended to the ciphertext in cryptography library; ensures integrity.

* Decryption: Use aesgcm.decrypt(nonce, ciphertext, aad); if the ciphertext or AAD is tampered, an exception is raised.

In [18]:
print("Key:", key.hex())
print("Nonce:", nonce.hex())

Key: 53ecdbe519bb4f77255acfb496f9cc13
Nonce: 16b60faf00ef537aa2ed36b2


In [19]:
print("Plaintext :", plaintext)

Plaintext : b'A un amigo perdido'


In [20]:
print("Plaintext (hex):", plaintext.hex())

Plaintext (hex): 4120756e20616d69676f207065726469646f


In [21]:
print(' '.join(format(b, '08b') for b in plaintext))

01000001 00100000 01110101 01101110 00100000 01100001 01101101 01101001 01100111 01101111 00100000 01110000 01100101 01110010 01100100 01101001 01100100 01101111


In [22]:
print("Ciphertext (hex):", ciphertext.hex())

Ciphertext (hex): 4f43cb6dd1be57a68ba8eb01801946169f0611999629433f801d209cfc1343e3ba55


In [23]:
print(' '.join(format(b, '08b') for b in ciphertext))

01001111 01000011 11001011 01101101 11010001 10111110 01010111 10100110 10001011 10101000 11101011 00000001 10000000 00011001 01000110 00010110 10011111 00000110 00010001 10011001 10010110 00101001 01000011 00111111 10000000 00011101 00100000 10011100 11111100 00010011 01000011 11100011 10111010 01010101


In [24]:
# 5. Decrypt (will raise exception if tampered)
decrypted = aesgcm.decrypt(nonce, ciphertext, aad)
print("Decrypted:", decrypted.decode())

Decrypted: A un amigo perdido
